# 🧪 Task 3: A/B Hypothesis Testing on Risk Drivers
This notebook statistically validates risk drivers using Claim Frequency, Claim Severity, and Margin. We test four key hypotheses about differences in risk across regions and genders.

## 1. Metric Selection
- **Claim Frequency** = # claims / # policies
- **Claim Severity** = total claims / # claims
- **Margin** = TotalPremium - TotalClaims

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Metric calculations
df['HasClaim'] = df['TotalClaims'] > 0
df['ClaimFrequency'] = df['HasClaim'].astype(int)
df['ClaimSeverity'] = np.where(df['HasClaim'], df['TotalClaims'], np.nan)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

## 2. Test 1: Risk Difference Across Provinces

In [ ]:
province_groups = df[['Province', 'ClaimFrequency', 'ClaimSeverity', 'Margin']].dropna()
anova_freq = stats.f_oneway(*[g['ClaimFrequency'] for _, g in province_groups.groupby('Province')])
anova_sev = stats.f_oneway(*[g['ClaimSeverity'].dropna() for _, g in province_groups.groupby('Province')])

print("ANOVA p-value for Claim Frequency across Provinces:", anova_freq.pvalue)
print("ANOVA p-value for Claim Severity across Provinces:", anova_sev.pvalue)

## 3. Test 2: Risk Difference Between Zip Codes

In [ ]:
zip_groups = df[['PostalCode', 'ClaimFrequency', 'ClaimSeverity', 'Margin']].dropna()
top_zips = zip_groups['PostalCode'].value_counts().head(2).index
zip_test = zip_groups[zip_groups['PostalCode'].isin(top_zips)]

a = zip_test[zip_test['PostalCode'] == top_zips[0]]
b = zip_test[zip_test['PostalCode'] == top_zips[1]]

print("T-test on Claim Frequency:", stats.ttest_ind(a['ClaimFrequency'], b['ClaimFrequency']))
print("T-test on Claim Severity:", stats.ttest_ind(a['ClaimSeverity'], b['ClaimSeverity']))

## 4. Test 3: Margin Differences Between Zip Codes

In [ ]:
print("T-test on Margin:", stats.ttest_ind(a['Margin'], b['Margin']))

## 5. Test 4: Risk Differences Between Women and Men

In [ ]:
gender_data = df[df['Gender'].isin(['Male', 'Female'])][['Gender', 'ClaimFrequency', 'ClaimSeverity']]
g_male = gender_data[gender_data['Gender'] == 'Male']
g_female = gender_data[gender_data['Gender'] == 'Female']

print("Claim Frequency Difference (t-test):", stats.ttest_ind(g_male['ClaimFrequency'], g_female['ClaimFrequency']))
print("Claim Severity Difference (t-test):", stats.ttest_ind(g_male['ClaimSeverity'].dropna(), g_female['ClaimSeverity'].dropna()))

## 6. Interpretation and Business Recommendations
Use p-values to determine if we reject each null hypothesis.

Examples:
- ✅ Reject province H₀ if p < 0.05 ➝ Adjust premium by region.
- ✅ Reject gender H₀ if p < 0.05 ➝ Consider sex-based underwriting if allowed.
- ❌ Fail to reject ➝ No pricing changes required for that feature.